# Spanning trees

Anton Antonov   
July 2024

----

## Setup

In [1]:
use Graph;

use Graph::Circulant;
use Graph::Complete;
use Graph::CompleteKaryTree;
use Graph::Cycle;
use Graph::Grid;
use Graph::Hypercube;
use Graph::KnightTour;
use Graph::Star;
use Graph::Path;
use Graph::Petersen;
use Graph::Wheel;

use Graph::Distribution;
use Graph::Random;

use Data::Reshapers;
use Data::Summarizers;
use Data::TypeSystem;
use Data::Translators;
use Data::Geographics;
use Math::DistanceFunctions;
use Math::Nearest;
use Hash::Merge;

use JavaScript::D3;
use WWW::MermaidInk;

In [2]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

In [3]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

In [4]:
my $title-color = 'Silver';
my $stroke-color = 'SlateGray';
my $tooltip-color = 'LightBlue';
my $tooltip-background-color = 'none';
my $background = '1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $vertex-size = 6;
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

my %opts = :$background, :$title-color, :$edge-thickness, :$vertex-size;

{background => 1F1F1F, edge-thickness => 3, title-color => Silver, vertex-size => 6}

-----

## Small Geo-points set

A company is planning a fiber network for a number of Chicago suburbs. It only has the right of way for its fiber along certain corridors. Some of those corridors might be more expensive. Find the subgraph of connection corridors that connect every suburb with the lowest total cost:

In [5]:
my @dsEdges =
        %('from'=>'Maywood','to'=>'Downers','weight'=>14),
        %('from'=>'Maywood','to'=>'Oakbrook','weight'=>28),
        %('from'=>'Maywood','to'=>'Addison','weight'=>10),
        %('from'=>'Downers','to'=>'Grove','weight'=>25),
        %('from'=>'Downers','to'=>'Stickney','weight'=>23),
        %('from'=>'Downers','to'=>'Wheaton','weight'=>20),
        %('from'=>'Downers','to'=>'Oakbrook','weight'=>27),
        %('from'=>'Oakbrook','to'=>'Stickney','weight'=>11),
        %('from'=>'Oakbrook','to'=>'Addison','weight'=>17),
        %('from'=>'Addison','to'=>'McCook','weight'=>21),
        %('from'=>'Grove','to'=>'Wheaton','weight'=>10),
        %('from'=>'Stickney','to'=>'Wheaton','weight'=>15),
        %('from'=>'Stickney','to'=>'Bellwood','weight'=>20),
        %('from'=>'Wheaton','to'=>'Bellwood','weight'=>10),
        %('from'=>'Bellwood','to'=>'McCook','weight'=>28);

deduce-type(@dsEdges)

Vector(Struct([from, to, weight], [Str, Str, Int]), 15)

In [6]:
my $g = Graph.new(@dsEdges)

Graph(vertexes => 9, edges => 15, directed => False)

In [7]:
#% js
$g.edges(:dataset)
==> js-d3-graph-plot(
    width => 800,
    :$background, :$title-color, :$edge-thickness, :$vertex-size,
    force => {charge => { strength => -800}, collision => {radius => 50}}
    )

In [8]:
my $stree = $g.find-spanning-tree;

Graph(vertexes => 9, edges => 8, directed => False)

In [9]:
#% js
$g.edges
==> js-d3-graph-plot(
    highlight => $stree.edges,
    width => 800,
    :$background, :$title-color, :$edge-thickness, :$vertex-size,
    force => {charge => { strength => -800}, collision => {radius => 50}}
    )

-------

## Grid graph

In [10]:
my $g = Graph::Grid.new(n => 4, m => 6)

Graph(vertexes => 24, edges => 38, directed => False)

In [11]:
my $stree = $g.find-spanning-tree

Graph(vertexes => 24, edges => 23, directed => False)

In [12]:
#% js
$g.edges
==> js-d3-graph-plot(
    highlight => [|$stree.vertex-list, |$stree.edges],
    width => 500,
    :$background, :$title-color, :$edge-thickness, :$vertex-size,
    force => {charge => {strength => -160, iterations => 2}, collision => {radius => 4, iterations => 1}, link => {distance => 0}}
    )

-----

## Florida cities

Build an interstate highway system joining the cities of the state of Florida, USA.

Here is a dataset of large enough cities:

In [13]:
my @cities = city-data().grep({ $_<State> eq 'Florida' && $_<Population> ≥ 50_000 });
@cities.elems

78

In [14]:
#% html
@cities.head(4) ==> to-html

Elevation,Longitude,ID,City,LocationLink,Latitude,State,Country,Population
5,-81.6613021,United_States.Florida.Jacksonville,Jacksonville,"http://maps.google.com/maps?q=30.3370193,-81.6613021&z=12&t=h",30.3370193,Florida,United States,949611
2,-80.2086152,United_States.Florida.Miami,Miami,"http://maps.google.com/maps?q=25.775163,-80.2086152&z=12&t=h",25.775163,Florida,United States,442241
1,-82.4796734,United_States.Florida.Tampa,Tampa,"http://maps.google.com/maps?q=27.9700861,-82.4796734&z=12&t=h",27.9700861,Florida,United States,384959
30,-81.375789,United_States.Florida.Orlando,Orlando,"http://maps.google.com/maps?q=28.533513,-81.375789&z=12&t=h",28.533513,Florida,United States,307573


Here is the corresponding weighted graph:

In [15]:
my %coords = @cities.map({ $_<City> => ($_<Latitude>, $_<Longitude>) });
my @dsEdges = (%coords.keys X %coords.keys ).map({ %(from => $_.head, to => $_.tail, weight => geo-distance(|%coords{$_.head}, |%coords{$_.tail} )) })

[{from => Saint Petersburg, to => Saint Petersburg, weight => 0} {from => Saint Petersburg, to => Hialeah, weight => 313767.95377975825} {from => Saint Petersburg, to => Cape Coral, weight => 140044.59209728107} {from => Saint Petersburg, to => Sunrise, weight => 295578.27396797645} {from => Saint Petersburg, to => Pompano Beach, weight => 301372.28016257304} {from => Saint Petersburg, to => Fountainbleau, weight => 317742.15864710894} {from => Saint Petersburg, to => Palm Beach Gardens, weight => 265251.1719458123} {from => Saint Petersburg, to => Miami, weight => 327866.53706754773} {from => Saint Petersburg, to => Pembroke Pines, weight => 299750.4274562403} {from => Saint Petersburg, to => Homestead, weight => 336305.35833421163} {from => Saint Petersburg, to => Lakeland, weight => 75387.33615757653} {from => Saint Petersburg, to => Deltona, weight => 189497.72008852873} {from => Saint Petersburg, to => Miramar, weight => 307222.44374830154} {from => Saint Petersburg, to => West Pa

In [16]:
my $g = Graph.new(@dsEdges, :!directed)

Graph(vertexes => 78, edges => 3042, directed => False)

In [17]:
my $stree = $g.find-spanning-tree(method => 'kruskal')

Graph(vertexes => 78, edges => 77, directed => False)

In [18]:
#my $stree2 = $g.find-spanning-tree(method => 'prim')

()

In [32]:
#% js
$stree.edges
==> js-d3-graph-plot(
    width => 1200,
    height => 1000,
    :$background, :$title-color, :$edge-thickness, :$vertex-size,
    force => {charge => {strength => -300, iterations => 2}, collision => {radius => 4, iterations => 1}, link => {distance => 'EdgeWeight'}}
    )